In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime 
import sys
%matplotlib inline
evaluation=pd.read_csv('submit.txt')
product_info=pd.read_csv('product_info.txt')
product_quantity=pd.read_csv('product_quantity.txt')
old_data = pd.read_csv("train_month_t_week_product_info.csv")
p_evaluation = pd.read_csv("p_evaluation.csv")
old_data=old_data.rename(columns={'ciiquantity': 'ciiquantity_month'})
#['eval','eval2','eval3','eval4','voters','maxstock']
#['f0',' f1','f2','f3','f4','f5']
name={'f0':'eval','f1':'eval2','f2':'eval3','f3':'eval4','f4':'voters','f5':'maxstock'}

In [9]:
def regularization(table,name):
    mean, std = table[name].mean(), table[name].std()
    table.loc[:, name] = (table[name] - mean)/std
    return table

In [11]:
for x in ['eval4', 'voters' ,'maxstock']:
    product_info=regularization(product_info,x)
product_info.head()

,product_id,district_id1,district_id2,district_id3,district_id4,lat,lon,railway,airport,citycenter,...,citycenter2,eval,eval2,eval3,eval4,voters,startdate,upgradedate,cooperatedate,maxstock
0,1,10201,20502,31003,45760,3.994928,11.634630,-1.0,-1.0,-1.0,...,-1.0,3.0,5,2.0,-0.830884,-0.266098,2005-11-01,2015-01-01,2013-07-02,-0.529165
1,2,10201,20502,31003,45760,3.995148,11.636258,-1.0,-1.0,2.3,...,-1.0,3.0,4,2.0,-0.111140,0.063709,2005-02-28,2011-01-01,2014-12-16,0.020579
2,3,10201,20502,31003,45760,3.994291,11.631246,-1.0,-1.0,10.3,...,-1.0,3.0,4,2.5,0.368689,0.079391,2007-03-01,2014-01-01,2014-07-02,0.111258
3,4,10201,20502,31003,55952,3.997783,11.641561,-1.0,-1.0,9.3,...,-1.0,3.0,4,2.5,0.128774,-0.250906,2006-07-01,1753-01-01,2014-12-19,-0.296799
4,5,10201,20502,31003,55952,3.999904,11.641149,12.1,25.2,13.3,...,3.6,3.0,5,2.0,-0.111140,0.309717,2007-01-01,2012-03-01,2007-11-07,-0.415815


In [2]:
def one_hot(table,name):
    dummies = pd.get_dummies(table[name], prefix=name, drop_first=False)
    table = pd.concat([table, dummies], axis=1)
    table.drop(name, axis=1)
    return table

In [3]:
for x in ['district_id1', 'district_id2']:
    product_info=one_hot(product_info,x)
print(product_info.columns)

Index(['product_id', 'district_id1', 'district_id2', 'district_id3',
       'district_id4', 'lat', 'lon', 'railway', 'airport', 'citycenter',
       ...
       'district_id2_1148418', 'district_id2_1150458', 'district_id2_1151274',
       'district_id2_1151376', 'district_id2_1153722', 'district_id2_1157394',
       'district_id2_1160352', 'district_id2_1187790', 'district_id2_1193808',
       'district_id2_1307640'],
      dtype='object', length=104)


In [4]:
product_info_use=product_info.drop(['district_id1', 'district_id4', 'district_id3', 'district_id2', 'lat', 'lon', 'railway'
                                   , 'airport', 'citycenter', 'railway2', 'citycenter2', 'eval', 'eval2', 'eval3', 'eval4'
                                   , 'voters', 'startdate', 'upgradedate', 'cooperatedate', 'maxstock'],axis=1)

In [12]:
old_data

,ciiquantity_month,product_id,product_month,w0,w1,w2,w3,w4,w5,w6,...,month_09,month_10,month_11,month_12,f0,f1,f2,f3,f4,f5
0,29.0,1.0,2014-01,4.0,4.0,5.0,5.0,5.0,4.0,4.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
1,111.0,1.0,2014-02,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
2,13.0,1.0,2014-03,5.0,4.0,4.0,4.0,4.0,5.0,5.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
3,71.0,1.0,2014-04,4.0,5.0,5.0,4.0,4.0,4.0,4.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
4,74.0,1.0,2014-05,4.0,4.0,4.0,5.0,5.0,5.0,4.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
5,30.0,1.0,2014-06,5.0,4.0,4.0,4.0,4.0,4.0,5.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
6,55.0,1.0,2014-07,4.0,5.0,5.0,5.0,4.0,4.0,4.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
7,159.0,1.0,2014-08,4.0,4.0,4.0,4.0,5.0,5.0,5.0,...,0,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
8,35.0,1.0,2014-09,5.0,5.0,4.0,4.0,4.0,4.0,4.0,...,1,0,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165
9,134.0,1.0,2014-10,4.0,4.0,5.0,5.0,5.0,4.0,4.0,...,0,1,0,0,-0.800491,0.261345,-0.712015,-0.830884,-0.266098,-0.529165


In [5]:
def chose_train(table):
    train_month_t_week_product_info = table
    train_month_t_week_product_info["year"]=train_month_t_week_product_info["product_month"].apply(lambda x:x[0:4])
    train_month_t_week_product_info["year_s"]=train_month_t_week_product_info["year"].apply(lambda x:(float(x)-2015.5)/4)
    train_month_t_week_product_info["w0"]=train_month_t_week_product_info["w0"].apply(lambda x:x/10)
    train_month_t_week_product_info["w1"]=train_month_t_week_product_info["w1"].apply(lambda x:x/10)
    train_month_t_week_product_info["w2"]=train_month_t_week_product_info["w2"].apply(lambda x:x/10)
    train_month_t_week_product_info["w3"]=train_month_t_week_product_info["w3"].apply(lambda x:x/10)
    train_month_t_week_product_info["w4"]=train_month_t_week_product_info["w4"].apply(lambda x:x/10)
    train_month_t_week_product_info["w5"]=train_month_t_week_product_info["w5"].apply(lambda x:x/10)
    train_month_t_week_product_info["w6"]=train_month_t_week_product_info["w6"].apply(lambda x:x/10)
    train_month_t_week_product_info=train_month_t_week_product_info.drop(['year', 'month', 'f0', 'f1', 'f2','w6', 'w5', 'w4', 'w3',
       'w2', 'w1', 'w0'],
                                                                         axis=1)
    return train_month_t_week_product_info

In [6]:
old_data1=chose_train(old_data)
old_data1=old_data1.rename(columns=name)

In [7]:
p_evaluation1=chose_train(p_evaluation)
p_evaluation1=p_evaluation1.rename(columns=name)

In [8]:
def movecol(colname,df):
    x = df[colname]
    df.drop(labels=[colname], axis=1,inplace = True)
    df.insert(0, colname, x)

In [9]:
old_data1.head()

,ciiquantity_month,product_id,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12,eval4,voters,maxstock,year_s
0,29.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
1,111.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
2,13.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
3,71.0,1.0,0,0,0,1,0,0,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375
4,74.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0,-0.830884,-0.266098,-0.529165,-0.375


In [10]:
columns=old_data1.columns
for col in columns:
    movecol(col,p_evaluation1)
    movecol(col,old_data1)

In [11]:
print(old_data1.columns)
print(p_evaluation1.columns)

Index(['year_s', 'maxstock', 'voters', 'eval4', 'month_12', 'month_11',
       'month_10', 'month_09', 'month_08', 'month_07', 'month_06', 'month_05',
       'month_04', 'month_03', 'month_02', 'month_01', 'product_id',
       'ciiquantity_month'],
      dtype='object')
Index(['year_s', 'maxstock', 'voters', 'eval4', 'month_12', 'month_11',
       'month_10', 'month_09', 'month_08', 'month_07', 'month_06', 'month_05',
       'month_04', 'month_03', 'month_02', 'month_01', 'product_id',
       'ciiquantity_month'],
      dtype='object')


In [12]:
old_data1=pd.merge(old_data1,product_info_use,on='product_id',how='left')
p_evaluation1=pd.merge(p_evaluation1,product_info_use,on='product_id',how='left')
old_data1=old_data1.drop('product_id',axis=1)
p_evaluation1=p_evaluation1.drop('product_id',axis=1)

In [13]:
def random_read_train_data(table):    
    _features = table.drop('ciiquantity_month',axis=1).as_matrix()
    _targets = table['ciiquantity_month'].as_matrix()
    _targets.shape = (_targets.shape[0], 1)
    _targets.transpose()
    return _features,_targets

old_data1_features,old_data1_targets = random_read_train_data(old_data1)
p_evaluation1_features,p_evaluation1_targets = random_read_train_data(p_evaluation1)
print(old_data1_features.shape,old_data1_targets.shape)
print(p_evaluation1_features.shape,p_evaluation1_targets.shape)

(60557, 99) (60557, 1)
(56000, 99) (56000, 1)


In [14]:
def add_layer(inputs, in_size, out_size, activation_function=None):    
    Weights = tf.Variable(tf.random_normal([in_size, out_size],mean=0.0,stddev=0.1))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
in_shape=old_data1_features.shape[1]
hidden_shape1=50
hidden_shape2=25
x = tf.placeholder(tf.float32, shape=[None, in_shape])
y = tf.placeholder(tf.float32, [None,1])
l1 = add_layer(x, in_shape, hidden_shape1, activation_function=tf.nn.sigmoid)
l2 = add_layer(l1, hidden_shape1, hidden_shape2, activation_function=tf.nn.sigmoid)
prediction = add_layer(l2, hidden_shape2, 1, activation_function=None)

loss = tf.sqrt(tf.reduce_mean(tf.reduce_sum(tf.square(y - prediction),
                     reduction_indices=[1])))
init_op = tf.initialize_all_variables()
saver = tf.train.Saver()
sess=tf.Session()
sess.run(init_op)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [22]:
loss_value=sess.run(loss, feed_dict={x: old_data1_features, y: old_data1_targets})
print('Loss: {:.6f}'.format(loss_value))
epochs=3000
l_r=0.01
start = datetime.now()
train_step = tf.train.GradientDescentOptimizer(l_r).minimize(loss)
for epoch in range(epochs):    
    sess.run(train_step, feed_dict={x: old_data1_features, y: old_data1_targets})
    if epoch % 100 == 0:
        temploss = sess.run(loss, feed_dict={x: old_data1_features, y: old_data1_targets})
        end = datetime.now()
        sys.stdout.write('\rEpoch {:>2} Loss: {:.6f} time: {}'.format(epoch + 1,temploss,end-start))
        start = datetime.now()

Loss: 134.850266
Epoch 2901 Loss: 134.291336 time: 0:00:08.989893

In [23]:
ansow=sess.run(prediction, feed_dict={x: p_evaluation1_features})
ansow

array([[ 111.72888184],
       [ 152.1111145 ],
       [ 138.56297302],
       ..., 
       [  35.61442566],
       [  40.10958862],
       [ 151.31811523]], dtype=float32)

In [16]:
save_path = saver.save(sess, "./new_model_{}_{}.ckpt".format(in_shape,hidden_shape))
print(save_path)

./new_model_23_20.ckpt


In [24]:
evaluation.ciiquantity_month=ansow
evaluation.to_csv('nn_{}.csv'.format(loss_value),index=False)